In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import datetime

In [2]:
data_file_name = 'face_data.npz'
test_file_name = 'test_data.npz'

batch_size = 128

def init_weights(shape, name):
    """
    initialize weight
    shape: shape of weight
    name: weight name
    xavier 초기화 사용
    각각의 Filter를 만들어주는 함수
    """
    with tf.device('/gpu:0'):
        return tf.get_variable(name = name, shape = shape, initializer = tf.contrib.layers.xavier_initializer())

def init_bias(shape, name):
    """
    initialize bias
    bias를 만들어주는 함수
    """
    with tf.device('/gpu:0'):
        return tf.Variable(tf.constant(0.1, shape = shape))

def model(X, w1, w2, w3, w4, wo, b2, b3, b4, bo, p_keep_conv, p_keep_hidden):
    """
    create model
    X: first input
    w1, w2, w3, wo: weights for each layer
    b2, b3, bo: bias for each layer
    p_keep_conv, p_keep_hidden: constant for dropout
    
    모델생성 함수.
    총 3개의 Conv layers와 3개의 Max-pool layers, 2개의 FC layers로 구성
    Conv1 - Pooling1 - Conv2 - Pooling2 - Conv3 - Pooling3 - FC1 - FC2
    로 구성되어있고 각 풀링 이후와 첫번째 FC 이후에 dropout을 적용
    """
    
    with tf.device('/gpu:0'):
#         dw = tf.nn.max_pool(X, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME') 

        # First Conv layer. l1a shape = (?, 128, 128, 32)
        with tf.name_scope("layer1"):
            l1a = tf.nn.relu(tf.nn.conv2d(X, w1, strides = [1, 1, 1, 1], padding = 'SAME'))
            # First Max-pool layer. l1 shape = (?, 64, 64, 32)
            l1 = tf.nn.max_pool(l1a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')        
            l1 = tf.nn.dropout(l1, p_keep_conv)

        with tf.name_scope("layer2"):
            # l2a shape = (?, 64, 64, 64)
            l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides = [1, 1, 1, 1], padding = 'SAME') + b2)
            # l2 shape = (?, 32, 32, 64)
            l2 = tf.nn.max_pool(l2a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            l2 = tf.nn.dropout(l2, p_keep_conv)

        with tf.name_scope("layer3"):
            # l2a shape = (?, 64, 64, 64)
            l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides = [1, 1, 1, 1], padding = 'SAME') + b3)
            # l2 shape = (?, 32, 32, 64)
            l3 = tf.nn.max_pool(l3a, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
            print(l3.shape)
            l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
            l3 = tf.nn.dropout(l3, p_keep_conv)

        with tf.name_scope("layer4"):
            # reshape to ( , 3072)
            l4 = tf.nn.relu(tf.matmul(l3, w4) + b4)
            l4 = tf.nn.dropout(l4, p_keep_hidden)
            l4 = tf.matmul(l4, wo) + bo
            return l4

In [3]:
# 데이터 읽어옴
if os.path.isfile(data_file_name):
    file = np.load(data_file_name)
    data = file['data']
    labels = file['sex_labels']
else:
    print("No data file.")
    sys.exit()
    
if os.path.isfile(test_file_name):
    file = np.load(test_file_name)
    td = file['data']
    tl = file['sex_labels']
else:
    print("No test file")
    sys.exit()

print("data load success")

print(data.shape)
print(td.shape)

data load success
(1785, 128, 128, 1)
(166, 128, 128, 1)


In [4]:
tf.reset_default_graph()

# data, labels 를 위한 placeholder
X = tf.placeholder("float", [None, 128, 128, 1], name = "X")
Y = tf.placeholder("float", [None, 2], name = "Y")

# data, labels 를 위한 placeholder
w1 = init_weights([3, 3, 1, 32], "weight1") 
w2 = init_weights([3, 3, 32, 64], "weight2")
w3 = init_weights([3, 3, 64, 128], "weight3")
w4 = init_weights([16*16*128, 512], "weight4")
wo = init_weights([512, 2], "weight_out")

b2 = init_bias([64], "bias2")
b3 = init_bias([128], "bias3")
b4 = init_bias([512], "bias4")
bo = init_bias([2], "bias_out")

# tensorboard에 weights와 bias histogram 등록
tf.summary.histogram("weight1_summ", w1)
tf.summary.histogram("weight2_summ", w2)
tf.summary.histogram("bias2_summ", b2)
tf.summary.histogram("weight3_summ", w3)
tf.summary.histogram("bias3_summ", b3)
tf.summary.histogram("weight4_summ", w4)
tf.summary.histogram("bias4_summ", b4)
tf.summary.histogram("weight_out_summ", wo)
tf.summary.histogram("bias_out_summ", bo)

# dropout 위한 placeholder
p_keep_conv = tf.placeholder("float", name = "p_keep_conv")
p_keep_hidden = tf.placeholder("float", name = "p_keep_hidden")

# 모델생성
M = model(X, w1, w2, w3, w4, wo, b2, b3, b4, bo, p_keep_conv, p_keep_hidden)

# cost와 accuarcy를 계산해주는 함수 설정
with tf.name_scope("cost"):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = M, labels = Y))
    train_op = tf.train.RMSPropOptimizer(0.00001, 0.9).minimize(cost)
    tf.summary.scalar("cost", cost)

with tf.name_scope("accuarcy"):
    correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(M, 1))
    acc_op = tf.reduce_mean(tf.cast(correct_pred, "float"))
    tf.summary.scalar("accuracy", acc_op)

# 모델 저장을 위해 collection에 weight와 bias모음
tf.add_to_collection('vars', w1)
tf.add_to_collection('vars', w2)
tf.add_to_collection('vars', b2)
tf.add_to_collection('vars', w3)
tf.add_to_collection('vars', b3)
tf.add_to_collection('vars', w4)
tf.add_to_collection('vars', b4)
tf.add_to_collection('vars', wo)
tf.add_to_collection('vars', bo)

(?, 16, 16, 128)


In [5]:
# 모델 저장 경로 설정
save_path = "./models/"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# 세션 시작
with tf.Session(config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))) as sess:
    
    # tensorboard에 올림
    writer = tf.summary.FileWriter('./logs/face2', sess.graph)
    merged = tf.summary.merge_all()
    
    tf.global_variables_initializer().run()
    
    saver = tf.train.Saver()
    
    print(datetime.datetime.now().time())
    
    # epoch=450, batch size=128로 모델 학습
    for i in range(450):
        total_batch = int(len(data) / batch_size)
        training_batch = zip(range(0, len(data), batch_size), range(batch_size, len(data)+1, batch_size))
        li = np.zeros(1)
        for start, end in training_batch:
            c, _ = sess.run([cost, train_op], feed_dict = {X: data[start:end], Y: labels[start:end],
                                           p_keep_conv: 0.8, p_keep_hidden: 0.5})
            li[0] += c / total_batch
        summary, acc = sess.run([merged, acc_op], feed_dict = {X: td, Y: tl, p_keep_conv: 1.0, p_keep_hidden: 1.0})
        
        writer.add_summary(summary, i)
        print(i, li[0], acc)
    
    saved_path = saver.save(sess, os.path.join(save_path, 'face_recog_2.ckpt'))
    print("Checkpoint saved at ", saved_path)
    print(datetime.datetime.now().time())
    sess.close()

20:05:38.691027
0 13.2721449309 0.0783132
1 10.7473105593 0.0783133
2 7.49517394488 0.0843374
3 7.81770165207 0.0903614
4 5.93124557815 0.114458
5 6.08763130188 0.114458
6 5.66423679244 0.108434
7 4.55608227426 0.168675
8 5.21829625255 0.138554
9 4.64874382088 0.186747
10 3.52650795534 0.144578
11 3.77380854773 0.192771
12 3.74556110885 0.168675
13 3.07030856005 0.240964
14 2.96178135984 0.138554
15 2.1877068913 0.204819
16 1.94038394735 0.23494
17 2.07050676071 0.246988
18 1.69376654035 0.307229
19 1.64356003726 0.331325
20 1.23915267234 0.337349
21 1.2817294958 0.343374
22 1.06248817939 0.301205
23 1.05134177545 0.307229
24 0.882981113158 0.307229
25 0.819159989174 0.337349
26 0.743797441622 0.295181
27 0.679409009906 0.331325
28 0.695551010565 0.349398
29 0.751563520386 0.463855
30 0.64924338331 0.385542
31 0.739188133094 0.457831
32 0.62272910043 0.421687
33 0.56154904142 0.445783
34 0.5992943901 0.385542
35 0.566369408885 0.493976
36 0.603624911262 0.403614
37 0.60451424609 0.4819